In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import pickle
import statsmodels.api as sm

In [ ]:
folder = '/odinn/users/thjodbjorge/Proteomics/Mortality2/'
siblings = pd.read_csv('/odinn/users/thjodbjorge/islbok_siblings', delim_whitespace = True, header = None, names= ['PN','PN1','PN2','yob','yod','sex'], index_col='PN')
# Mortality = pd.read_csv('/odinn/users/thjodbjorge/MI/Mortality_risk_proteomics_20_90_wo_pregnant_13012020.txt', delim_whitespace=True,header=None, names=['PN','pred'], index_col = 'PN')
# Mortality = pd.read_csv(folder+'Mortality_risk_proteomics_60_105_29062020.txt',index_col='PN')
Mortality = pd.read_csv(folder+'Mortality_risk_proteomics_age_18plus_22092020.txt',index_col='PN')
pn_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/pn_info_2.csv',index_col = 'Barcode2d' )

file = open(folder+"Mor_keep_samples.pkl",'rb')
keep_samples_dict = pickle.load(file)


print(Mortality.shape)

In [ ]:
old_dataset = 'Old_18105'
new_dataset = 'New_18105'
keep_samples = keep_samples_dict[old_dataset].union(keep_samples_dict[new_dataset])
print(len(keep_samples))
# keep_samples = keep_samples_dict['All_60105']

Mortality = Mortality.loc[pn_info.loc[keep_samples,'PN']]
print(Mortality.shape)

pn_info = pn_info.reset_index().set_index('PN')

In [ ]:
display(pn_info.head())
display(siblings.head())
display(Mortality.head())


In [ ]:
Mortality = pd.merge(Mortality,pn_info[['Age_at_sample_collection_2','sex','yob']], left_index=True,right_index=True, how = 'left') 
display(Mortality.head())
Mortality['age'] = Mortality['Age_at_sample_collection_2']
Mortality['age2'] = Mortality['age']**2
Mortality['age3'] = Mortality['age']**3
Mortality['age4'] = Mortality['age']**4
Mortality['lnage'] = np.log(Mortality['age'])
Mortality['agesex'] = Mortality['age'] * (Mortality['sex']-1)
Mortality['age2sex'] = Mortality['age2'] * (Mortality['sex']-1)
Mortality['age3sex'] = Mortality['age3'] * (Mortality['sex']-1)
agesex = ['age','age2','agesex','age2sex','sex','age3','lnage','yob']
# agesex = ['age','age2','agesex','age2sex','sex','yob']

pred_agesex =  sm.OLS(Mortality['pred'].values,sm.add_constant(Mortality[agesex].values)).fit().predict(sm.add_constant(Mortality[agesex].values))
Mortality['corr_pred'] = Mortality['pred'] - pred_agesex
print(Mortality.shape)
Mortality.head()

In [ ]:
print(pearsonr(Mortality['corr_pred'],Mortality.age))
print(pearsonr(Mortality['pred'],Mortality.age))
print(pearsonr(Mortality['corr_pred'],Mortality.age2))
print(pearsonr(Mortality['pred'],Mortality.age2))
print(pearsonr(Mortality['corr_pred'],Mortality.age3sex))
print(pearsonr(Mortality['pred'],Mortality.age3sex))
print('')
print(pearsonr(Mortality['corr_pred'],Mortality.age3))
print(pearsonr(Mortality['pred'],Mortality.age3))
print(pearsonr(Mortality['corr_pred'],Mortality.lnage))
print(pearsonr(Mortality['pred'],Mortality.lnage))
print(pearsonr(Mortality['corr_pred'],Mortality.yob))
print(pearsonr(Mortality['pred'],Mortality.yob))
print(pearsonr(Mortality['corr_pred'],Mortality.age4))
print(pearsonr(Mortality['pred'],Mortality.age4))

In [ ]:
# Create matrix of all sibling pairs
sib_pred = pd.merge(siblings,Mortality, how='inner',left_index=True,right_index=True)
sib_pred['siblings'] = sib_pred.groupby(['PN1','PN2']).pred.transform('count')
sib_pred = sib_pred[sib_pred['siblings']>1].sort_values(['PN1','PN2'])
sib_pred['count_sib'] = sib_pred['siblings']

new_data = []
new_data_corr = []
for i,ind in enumerate(sib_pred.index):
    for j in range(1,sib_pred.loc[ind,'count_sib']):
        new_data.append([sib_pred.loc[ind,'pred'],sib_pred.iloc[i+j]['pred']])
        new_data_corr.append([sib_pred.loc[ind,'corr_pred'],sib_pred.iloc[i+j]['corr_pred']])
        indn=sib_pred.iloc[i+j].name
        sib_pred.loc[indn,'count_sib'] -= 1
#         print(sib_pred.iloc[i+j]['count_sib'])
new_data = np.array(new_data)
new_data_corr = np.array(new_data_corr)
print(sib_pred.shape)
print(new_data.shape)

In [ ]:
sib_pred.head(10)

In [ ]:
# Calculate correlation
print('Sibling pairs: ', new_data.shape[0])
print('Siblings: ' ,pearsonr(new_data[:,0],new_data[:,1]), pearsonr(new_data[:,0],new_data[:,1])[0]*2)
print('random samples: ', pearsonr(sib_pred.pred,sib_pred.sample(frac=1).pred), pearsonr(sib_pred.pred,sib_pred.sample(frac=1).pred)[0]*2)

print('Better corrected siblings: ' ,pearsonr(new_data_corr[:,0],new_data_corr[:,1]),pearsonr(new_data_corr[:,0],new_data_corr[:,1])[0]*2)



In [ ]:
islbok = pd.read_csv('/odinn/users/thjodbjorge/islbok_1900', delim_whitespace = True, header = None, names= ['PN','PN1','PN2','yob','yod','sex'])
islbok_pred = pd.merge(islbok,Mortality, how='inner',left_on='PN',right_index=True)

In [ ]:
isl1 = pd.merge(islbok_pred,islbok_pred,left_on = 'PN1', right_on = 'PN', how='inner')
isl2 = pd.merge(islbok_pred,islbok_pred,left_on = 'PN2', right_on = 'PN', how='inner')

In [ ]:
isl1.columns

In [ ]:
print(isl1.shape)
po_pred = pd.concat([isl1,isl2])
po_pred.shape

In [ ]:
print(po_pred.columns)
po_pred.head()

In [ ]:
print('Parent offspring pairs: ', po_pred.shape[0])
print('parent offspring correlation better corrected pred: ' , pearsonr(po_pred.corr_pred_x,po_pred.corr_pred_y), pearsonr(po_pred.corr_pred_x,po_pred.corr_pred_y)[0]*2)
print('parent offspring correlation original correction: ' , pearsonr(po_pred.pred_x,po_pred.pred_y),pearsonr(po_pred.pred_x,po_pred.pred_y)[0]*2)

In [ ]:
print('parent offspring: ' ,pearsonr(po_pred.pred_x,po_pred.yob_x_x))
print('parent offspring: ' ,pearsonr(po_pred.pred_x,po_pred.age_x))

In [ ]:
print('parent offspring: ' ,pearsonr(po_pred.corr_pred_x,po_pred.yob_x_x))
print('parent offspring: ' ,pearsonr(po_pred.corr_pred_y,po_pred.yob_x_y))
print('parent offspring: ' ,pearsonr(po_pred.pred_x,po_pred.yob_x_x))
print('parent offspring: ' ,pearsonr(po_pred.pred_y,po_pred.yob_x_y))
print('parent offspring: ' ,pearsonr(po_pred.corr_pred_x,po_pred.age_x))
print('parent offspring: ' ,pearsonr(po_pred.corr_pred_y,po_pred.age_y))
print('parent offspring: ' ,pearsonr(po_pred.pred_x,po_pred.age_x))
print('parent offspring: ' ,pearsonr(po_pred.pred_y,po_pred.age_y))

In [ ]:
print(pearsonr(sib_pred.age,sib_pred.pred))
print(pearsonr(sib_pred.yob_x,sib_pred.pred))
print(pearsonr(sib_pred.age2,sib_pred.pred))